# Tutorial

In [1]:
import torch
import numpy as np

First of all we need to create cuda extension:

In [2]:
from torch.utils import cpp_extension

cpp_extension.load(
    name="sparse_accumulation_cuda",
    sources=["cuda_optimized/sparse_accumulation_cuda_kernel2D.cu"],
    is_python_module=False,
    extra_cuda_cflags=None,
    verbose=True,
)

Using /home/pozdn/.cache/torch_extensions/py38_cu111 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/pozdn/.cache/torch_extensions/py38_cu111/sparse_accumulation_cuda/build.ninja...
Building extension module sparse_accumulation_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module sparse_accumulation_cuda...


The goal is to compute this for all $\mu$:

$\text{Output}[:, :, \mu] = \sum\limits_{m_1, m_2} \text{X_1}[:, :, m_1] * \text{X_2}[:, :, m_2] * C_{m_1, m_2, \mu}$

This is the subpart of the Clebsch-Gordan iteration for fixed l1, l2, and l. The first two dimensions are the "dense" ones, so the same operation is performed for all the indices in the first two dimensions. 

Since Clebsch-Gordan coefficients are very sparse, it is worthwhile to align them into a 1-dimensional tensor containing only non-zero values, but in this case, we need to supply this tensor with supplementary indices tensors telling us what are the corresponding m1, m2, and $\mu$ indices. 

Reference slow python implementation is as simple as this:

In [3]:
def sparse_accumulation_loops(X1, X2, idx_output, output_size, idx_1, idx_2, multipliers):
    device = X1.device #all tensors must be on the same device and blah, blah, blah    
    dtype = X1.dtype    
    
    output = torch.zeros([X1.shape[0], X2.shape[1], output_size], device = device,dtype=dtype)
    for index in range(idx_output.shape[0]):       
        output[:, :, idx_output[index]] += X1[:, :, idx_1[index]] * X2[:, :, idx_2[index]] * multipliers[index]
    return output

Here multipliers are the values of Clebsch-Gordan coefficients, idx_1 is the tensor containing corresponding m1 indices, idx_2 is the tensor containing corresponding m2 indices, and idx_output is the tensor containing $\mu$ indices. output_size is just a single integer, the desired length of the output (2 * l + 1). 

So the loops go over all the terms, for all $\mu$, m1, and m2 with non-zero clebsch-gordan coefficients, and the current contribution is added to the output array to the proper place defined by $\mu$ which is stored in the idx_output

The first two dense dimensions are introduced, keeping in mind batch and feature dimensions. If you need just 1, it is possible to introduce a dummy dimension of size 1 ^^. 


The transformation itself, i.e., Clebsch-Gordan coefficients, can be precomputed once at the very beginning. This repo also contains the code for this:

In [4]:
from clebsch_gordan import ClebschGordan, get_real_clebsch_gordan
L_MAX = 5
clebsch = ClebschGordan(L_MAX).precomputed_
indices = get_real_clebsch_gordan(clebsch[L_MAX, L_MAX, L_MAX], L_MAX, L_MAX, L_MAX)

m1_aligned, m2_aligned = [], []
multipliers, mu_aligned = [], []
for mu in range(0, 2 * L_MAX + 1):
    for el in indices[mu]:
        m1, m2, multiplier = el
        m1_aligned.append(m1)
        m2_aligned.append(m2)
        multipliers.append(multiplier)
        mu_aligned.append(mu)
m1_aligned = torch.LongTensor(m1_aligned)
m2_aligned = torch.LongTensor(m2_aligned)
mu_aligned = torch.LongTensor(mu_aligned)
multipliers = torch.FloatTensor(multipliers)

indices = np.argsort(mu_aligned)

m1_aligned = m1_aligned[indices].cuda()
m2_aligned = m2_aligned[indices].cuda()
mu_aligned = mu_aligned[indices].cuda()
multipliers = multipliers[indices].cuda()

print("L_MAX: ")
print("multipliers shape: ", multipliers.shape)
print("m1_aligned shape: ", m1_aligned.shape)
print("m2_aligned shape: ", m2_aligned.shape)
print("multipliers shape: ", multipliers.shape)

L_MAX: 
multipliers shape:  torch.Size([126])
m1_aligned shape:  torch.Size([126])
m2_aligned shape:  torch.Size([126])
multipliers shape:  torch.Size([126])


This is a simple wrapper on sympy package, and the definition of the real clebsch-gordan coefficients is consistent with librascal real spherical harmonics, nice, wigner iterations, and rascaline

Now we can do the Clebsch-Gordan iteration:

In [5]:
X1 = torch.randn(100, 17, 2 * L_MAX + 1).cuda()
X2 = torch.randn(100, 17, 2 * L_MAX + 1).cuda()

output_loops = sparse_accumulation_loops(X1, X2, mu_aligned, 2 * L_MAX + 1, m1_aligned, m2_aligned, multipliers)
print(output_loops.shape)

torch.Size([100, 17, 11])


The main contribution of this package is the very optimized cuda (and cpu) implementation of this operation which can be invoked like this:

In [6]:
output_fast = torch.ops.sparse_accumulation_cuda.forward(X1, X2, mu_aligned, 2 * L_MAX + 1, m1_aligned, m2_aligned, multipliers)[0]
print(output_fast.shape)

torch.Size([100, 17, 11])


outputs are the same:

In [7]:
print(torch.max(torch.abs(output_loops - output_fast)))

tensor(5.9605e-07, device='cuda:0')


You can take a look at the benchmarks files .py along with their output .out to get an idea 1) how to benchmark this properly with gpu synchronization and the speed of this operation compared to a naive implementation